In [47]:
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict
import time
import pandas as pd

## Getting the data

We are getting the data beginning from 1980-81 season up to past season for train and validation.
We are beginning from 1980-81 season because previous to that season, voting was done by players.

In [60]:
def work_player_profile(param, season):
    url = "https://www.basketball-reference.com" + param
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    print(url)
    
    data_dict = {}
    
    per_game = soup.find(attrs={'id': 'all_per_game'})
    for row in per_game.findAll("tr"):
        if 'id' in row.attrs and row.attrs['id'] == 'per_game.' + season:
            print(float(row.find('td', attrs={'data-stat': 'fga_per_g'}).text))
            data_dict['fga'] = float(row.find('td', attrs={'data-stat': 'fga_per_g'}).text)
            data_dict['fg3a'] = float(row.find('td', attrs={'data-stat': 'fg3a_per_g'}).text)
            data_dict['fta'] = float(row.find('td', attrs={'data-stat': 'fta_per_g'}).text)
            break
    
    advanced_table = soup.find(attrs={'id': 'all_advanced'})
    for child in advanced_table.children:
        if 'table_outer_container' in child:
            other_soup = BeautifulSoup(child)
            rows = other_soup.findAll('tr')
    for row in rows:
        if 'id' in row.attrs and row.attrs['id'] == 'advanced.' + season:
            data_dict.update(
                {
                    'per': float(row.find('td', attrs={'data-stat': 'per'}).text),
                    'ts_pct': float(row.find('td', attrs={'data-stat': 'ts_pct'}).text),
                    'usg_pct': float(row.find('td', attrs={'data-stat': 'usg_pct'}).text),
                    'bpm': float(row.find('td', attrs={'data-stat': 'bpm'}).text),
                    'season': str(int(season)-1) + '-' + season[-2:],
                }
            )
            return data_dict

In [44]:
def get_stats_of_voting(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    item = soup.find(attrs={'class': 'stats_table'})
    rows = item.findAll('tr')
    
    season = url.split('.html')[0][-4:]
    print(f"Season: {season}")
    
    players_stats = defaultdict(list)
    
    for index, row in enumerate(rows):
        print(f"\fIndex: {index} of {len(rows)}")
        header_cells = row.findAll('th')
        for header_cell in header_cells:
            if ('data-stat' in header_cell.attrs and 
                header_cell['data-stat'] == 'ranker' and
                'csk' in header_cell.attrs):
                rank = int(header_cell.getText())
                print(rank)
        td_cells = row.findAll('td')
        if not td_cells:
            continue
        for cell in td_cells:
            if 'data-stat' not in cell.attrs:
                continue
            if cell['data-stat'] == 'age':
                continue
            elif cell['data-stat'] == 'team_id':
                base = "https://www.basketball-reference.com"
                try:
                    link = cell.find("a")['href']
                except Exception:
                    players_stats['win_pct'].append(0.5) # average
                    continue
                url = base + link
                time.sleep(1)
                soup = BeautifulSoup(requests.get(url).text)
                for item in soup.findAll('p'):
                    if "Record" in item.text:
                        record = re.findall('\d+\-\d+', item.text)[0]
                        splitted = record.split('-')
                        players_stats['win_pct'].append(float(splitted[0]) / (float(splitted[1]) + float(splitted[0])))
                        break
                continue
            #print('Text: ', cell.getText())
            #print(cell['data-stat'])
            if cell['data-stat'] == 'player':
                time.sleep(1)
                advanced_dict = work_player_profile(cell.find('a')['href'], season)
                for key in advanced_dict:
                    players_stats[key].append(advanced_dict[key])
                players_stats[cell['data-stat']].append(cell.getText())
            else:
                text = cell.getText() or "0"
                players_stats[cell['data-stat']].append(float(text))
    return players_stats

In [61]:
seasons = [2011]
new_data = defaultdict(list)

for season in seasons:
    url = f"https://www.basketball-reference.com/awards/awards_{str(season)}.html"
    season_dict = get_stats_of_voting(url)
    for key in season_dict:
        new_data[key].extend(season_dict[key])
data_frame = pd.DataFrame(new_data)
print(data_frame)

Season: 2011
Index: 0 of 15
Index: 1 of 15
Index: 2 of 15
https://www.basketball-reference.com/players/r/rosede01.html
19.7


KeyboardInterrupt: 

In [50]:
data_frame.head()

,fga,fg3a,fta,per,ts_pct,usg_pct,bpm,season,player,win_pct,...,pts_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,fg_pct,fg3_pct,ft_pct,ws,ws_per_48
0,19.7,4.8,6.9,23.5,0.550,32.2,5.9,2010-11,Derrick Rose,0.756098,...,25.0,4.1,7.7,1.0,0.6,0.445,0.332,0.858,13.1,0.208
1,13.4,0.1,11.7,26.1,0.616,27.2,4.8,2010-11,Dwight Howard,0.634146,...,22.9,14.1,1.4,1.4,2.4,0.593,0.000,0.596,14.4,0.235
2,18.8,3.5,8.4,27.3,0.594,31.5,8.6,2010-11,LeBron James,0.707317,...,26.7,7.5,7.0,1.6,0.6,0.510,0.330,0.759,15.6,0.244
3,20.0,4.3,7.1,23.9,0.548,35.1,3.8,2010-11,Kobe Bryant,0.695122,...,25.3,5.1,4.7,1.2,0.1,0.451,0.323,0.828,10.3,0.178
4,19.7,5.3,8.7,23.6,0.589,30.6,2.9,2010-11,Kevin Durant,0.670732,...,27.7,6.8,2.7,1.1,1.0,0.462,0.350,0.880,12.0,0.189


In [51]:
data_frame.columns

Index(['fga', 'fg3a', 'fta', 'per', 'ts_pct', 'usg_pct', 'bpm', 'season',
       'player', 'win_pct', 'votes_first', 'points_won', 'points_max',
       'award_share', 'g', 'mp_per_g', 'pts_per_g', 'trb_per_g', 'ast_per_g',
       'stl_per_g', 'blk_per_g', 'fg_pct', 'fg3_pct', 'ft_pct', 'ws',
       'ws_per_48'],
      dtype='object')

In [52]:
len(data_frame.columns)

26

In [57]:
data_frame.iloc[0]

fga                    19.7
fg3a                    4.8
fta                     6.9
per                    23.5
ts_pct                 0.55
usg_pct                32.2
bpm                     5.9
season              2010-11
player         Derrick Rose
win_pct            0.756098
votes_first             113
points_won             1182
points_max             1210
award_share           0.977
g                        81
mp_per_g               37.4
pts_per_g                25
trb_per_g               4.1
ast_per_g               7.7
stl_per_g                 1
blk_per_g               0.6
fg_pct                0.445
fg3_pct               0.332
ft_pct                0.858
ws                     13.1
ws_per_48             0.208
Name: 0, dtype: object